# Demand Forecasting (AI, Data Innovation Team, CJ Express)

## File: Training Model (Cluster Model)

 C.J. Express Group Co.,Ltd. All Rights Reserved.
 
 Author: Kao Panboonyuen, Ph.D.
 
 Year: 2021
 ![](https://github.com/kaocj/demandforecasting_darkside/raw/main/demand_main.PNG)

In [11]:
!rm -rf cjdarkside.py
!wget https://github.com/kaocj/demandforecasting_darkside/raw/main/cjdarkside.py -q --show-progress

cjdarkside.py       100%[===================>]  45.48K  --.-KB/s    in 0.001s  


In [6]:
spark

# Import Library

In [14]:
from cjdarkside import *

# Name your CJ DARKSIDE model

In [15]:
version = '1small'
versionFeaturePL = "PL1small"

gs_model = "gs://cj-demand-forecasting-dataset/model/2021-April-v1/"
gs_csv = "gs://cj-demand-forecasting-dataset/csv/"
gs_csv_splite = "gs://cj-demand-forecasting-dataset/csv_split/"

PL_FEATURES_PATH = gs_model+"demand_cjdarkside_pl_features_2021_v"+str(versionFeaturePL)+".plmodel"

PL_Bangkok_MODEL_Classification_PATH = gs_model+"demand_cjdarkside_pl_bkk_c_binary_model_2021_v"+str(version)+".plmodel"
PL_Bangkok_MODEL_Regression_PATH = gs_model+"demand_cjdarkside_pl_bkk_r_binary_model_2021_v"+str(version)+".plmodel"

PL_NotBangkok_MODEL_Classification_PATH = gs_model+"demand_cjdarkside_pl_not_bkk_c_binary_model_2021_v"+str(version)+".plmodel"
PL_NotBangkok_MODEL_Regression_PATH = gs_model+"demand_cjdarkside_pl_not_bkk_r_binary_model_2021_v"+str(version)+".plmodel"
gs_csv = "gs://cj-demand-forecasting-dataset/csv/"
gs_csv_splite = "gs://cj-demand-forecasting-dataset/csv_split/"

PL_FEATURES_PATH = gs_model+"demand_cjdarkside_pl_features_2021_v"+str(versionFeaturePL)+".plmodel"

PL_Bangkok_MODEL_Classification_PATH = gs_model+"demand_cjdarkside_pl_bkk_c_binary_model_2021_v"+str(version)+".plmodel"
PL_Bangkok_MODEL_Regression_PATH = gs_model+"demand_cjdarkside_pl_bkk_r_binary_model_2021_v"+str(version)+".plmodel"

PL_NotBangkok_MODEL_Classification_PATH = gs_model+"demand_cjdarkside_pl_not_bkk_c_binary_model_2021_v"+str(version)+".plmodel"
PL_NotBangkok_MODEL_Regression_PATH = gs_model+"demand_cjdarkside_pl_not_bkk_r_binary_model_2021_v"+str(version)+".plmodel"

In [16]:
select_variables_initial = ['cj_darkside_id','SalDate','BranchCode','MaterialCode','avgPriceDis',\
                            'avgPrice','supPrice','label','totalNetSale',\
                            'TotalQtySale','types','ZipCode','ProvinceEN','DistrictEN',\
                            'DayofMonth','Yearday','Month','DayofWeek','Year',\
                            'Quarter','WeekOfYear','MonthQuarter','Branch','Name']

select_variables_initial = ['cj_darkside_id','SalDate','BranchCode','MaterialCode','avgPriceDis',\
                            'avgPrice','supPrice',\
                            'types','ZipCode','ProvinceEN','DistrictEN',\
                            'DayofMonth','Yearday','Month','DayofWeek','Year',\
                            'Quarter','WeekOfYear','MonthQuarter','Branch','Name']

# Data Set (Input)

In [17]:
DATASET_PATH = "gs://cj-demand-forecasting-dataset/demand_prediction_pipeline/all_sku_2020_v3/part-0*"
DATASET_PATH_JUST_SPLIT = gs_csv_splite+'sku_v3_split*'
DATASET_PATH_JUST_ONE = "gs://cj-demand-forecasting-dataset/demand_prediction_pipeline/all_sku_2020_v3/part-00000-4827a120-7099-4e1f-bc07-b8f3352c2f12-c000.csv"
TEST_PATH_JUST_ONE = "gs://cj-demand-forecasting-dataset/demand_prediction_pipeline/all_sku_2020_v3/part-00096-4827a120-7099-4e1f-bc07-b8f3352c2f12-c000.csv"

In [18]:
TEST_PATH_JUST_TWO = "gs://cj-demand-forecasting-dataset/demand_prediction_pipeline/test_debug"

In [19]:
# TEST_PATH_BKK = "gs://cj-demand-forecasting-dataset/demand_prediction_pipeline/all_sku_2020_v3/_pmind01/df_bkk.csv"
# TEST_PATH_NOTBKK = "gs://cj-demand-forecasting-dataset/demand_prediction_pipeline/all_sku_2020_v3/_pmind01/df_not_bkk.csv"

# TEST_PATH_BKK = spark.read.format("csv").option("header", "true").load(TEST_PATH_BKK)
# TEST_PATH_NOTBKK = spark.read.format("csv").option("header", "true").load(TEST_PATH_NOTBKK)

In [20]:
branch_location = 'gs://cj-demand-forecasting-dataset/_additional/BranchDistrictTH_EN_2.csv'
welfare_location = 'gs://cj-demand-forecasting-dataset/_additional/csv_branch_welfare.csv'
reference_qty = 'gs://cj-demand-forecasting-dataset/_reference2/reference00*'
flag_qty = 'gs://cj-demand-forecasting-dataset/_additional/flag_sku_branch.csv'
price_sens = "gs://cj-demand-forecasting-dataset/ad_data/ps_agg_v2_0*"
tier_cj = "gs://cj-demand-forecasting-dataset/tier/tier00000*"

# CREATE TRAIN SET

In [21]:
def clean_basic2(df):
    #df = df.na.fill('NORMAL',subset=['types'])
    df = df.dropna(how='all')
    df = df.dropna()

    df = df.withColumn("BranchCode", df["BranchCode"].cast("integer"))
    df = df.withColumn("MaterialCode", df["MaterialCode"].cast('integer'))

    df = df.withColumn("avgPriceDis", df["avgPriceDis"].cast('double'))
    df = df.withColumn("avgPrice", df["avgPrice"].cast('double'))
    df = df.withColumn("supPrice", df["supPrice"].cast('double'))

    df = df.cache()

    return df

In [22]:
df = spark.read.format("csv").option("header", "true").load(TEST_PATH_JUST_TWO)
df = clean_basic2(df)

print('-----------------------------------')
print('-- welcome to the darkside of cj --')
print('-----------------------------------')

-----------------------------------
-- welcome to the darkside of cj --
-----------------------------------


In [23]:
df.show(2)

+----------+------------+----------+-----------+--------+--------+------+--------------------+-------------------+
|BranchCode|MaterialCode|   SalDate|avgPriceDis|avgPrice|supPrice| types|                Name|             Branch|
+----------+------------+----------+-----------+--------+--------+------+--------------------+-------------------+
|         2|    20026655|2021-10-03|       21.0|    21.0|    21.0|NORMAL|บ้านอ้อยน้ำตาลทรา...|สาขาที่00013 ท่าวัด|
|        24|    20013220|2021-10-03|       40.0|    40.0|    40.0|NORMAL|ซันไลต์ ล้างจานเล...|สาขาที่00033 สามชุก|
+----------+------------+----------+-----------+--------+--------+------+--------------------+-------------------+
only showing top 2 rows



In [24]:
df.printSchema()

root
 |-- BranchCode: integer (nullable = true)
 |-- MaterialCode: integer (nullable = true)
 |-- SalDate: string (nullable = true)
 |-- avgPriceDis: double (nullable = true)
 |-- avgPrice: double (nullable = true)
 |-- supPrice: double (nullable = true)
 |-- types: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Branch: string (nullable = true)



# Start making Features

In [25]:
# Step1) Create Feature: Location

branch_location_df = spark.read.format("csv").option("header", "true").load(branch_location)
branch_location_df = branch_location_df.withColumnRenamed("BranchCode","BranchCodeLocation")

df = df.join(branch_location_df, df.BranchCode == branch_location_df.BranchCodeLocation)

# Step2) Create Feature: All Play with Date and Welfare

df = df.select("*").withColumn("cj_darkside_id", monotonically_increasing_id())

df = df.withColumnRenamed("Date","SalDate")
df = df.withColumn('DayofMonth', F.dayofmonth(F.col('SalDate')))

df = extract_date(df)

df = select_variable(df, select_variables_initial)

wf = read_file(welfare_location,'csv')
df = transform_welfare_flag(df, wf)
df = transform_welfare_flag_day_2(df)

df = df.withColumnRenamed("types","promotion")

df = df.dropna(how='all')

In [26]:
df.columns

['cj_darkside_id',
 'SalDate',
 'BranchCode',
 'MaterialCode',
 'avgPriceDis',
 'avgPrice',
 'supPrice',
 'promotion',
 'ZipCode',
 'ProvinceEN',
 'DistrictEN',
 'DayofMonth',
 'Yearday',
 'Month',
 'DayofWeek',
 'Year',
 'Quarter',
 'WeekOfYear',
 'MonthQuarter',
 'Branch',
 'Name',
 'welfareFlag',
 'welfareFlagDay']

# Feature: ONE HOT ENCODER

In [27]:
# load features pipeline
pipeline_features = Pipeline.read().load(PL_FEATURES_PATH)
df_transformed = pipeline_features.fit(df).transform(df)

# Main (Feature Engineering)

In [28]:
# Show your previous darkside feature -> Here !!!

In [29]:
def clean_binary_2(df):
    #df = df.na.fill('NORMAL',subset=['types'])
    df = df.dropna(how='all')
    df = df.dropna()

    df = df.withColumn("BranchCode", df["BranchCode"].cast("integer"))
    df = df.withColumn("MaterialCode", df["MaterialCode"].cast('integer'))

    df = df.withColumn("Year", df["Year"].cast('integer'))

    df = df.withColumn("avgPriceDis", df["avgPriceDis"].cast('double'))
    df = df.withColumn("avgPrice", df["avgPrice"].cast('double'))
    df = df.withColumn("supPrice", df["supPrice"].cast('double'))

    df = df.withColumn("welfareFlag", df["welfareFlag"].cast('integer'))
    df = df.withColumn("welfareFlagDay", df["welfareFlagDay"].cast('integer'))
    df = df.withColumn("ZipCode", df["ZipCode"].cast('integer'))

    df = df.cache()

    return df

In [30]:
final_df = clean_binary_2(df_transformed)
final_df.cache()

DataFrame[cj_darkside_id: bigint, SalDate: string, BranchCode: int, MaterialCode: int, avgPriceDis: double, avgPrice: double, supPrice: double, promotion: string, ZipCode: int, ProvinceEN: string, DistrictEN: string, DayofMonth: int, Yearday: int, Month: int, DayofWeek: int, Year: int, Quarter: int, WeekOfYear: int, MonthQuarter: int, Branch: string, Name: string, welfareFlag: int, welfareFlagDay: int, BranchCode_idx: double, BranchCode_vec: vector, MaterialCode_idx: double, MaterialCode_vec: vector, promotion_idx: double, promotion_vec: vector, ProvinceEN_idx: double, ProvinceEN_vec: vector, DistrictEN_idx: double, DistrictEN_vec: vector]

# Feature: High-Q

In [31]:
def clean_binary_4(df):
    #df = df.na.fill('NORMAL',subset=['types'])
    df = df.dropna(how='all')
    df = df.dropna()

    df = df.withColumn("BranchCode", df["BranchCode"].cast("integer"))
    df = df.withColumn("MaterialCode", df["MaterialCode"].cast('integer'))

    df = df.withColumn("Year", df["Year"].cast('integer'))

    df = df.withColumn("avgPriceDis", df["avgPriceDis"].cast('double'))
    df = df.withColumn("avgPrice", df["avgPrice"].cast('double'))
    df = df.withColumn("supPrice", df["supPrice"].cast('double'))

    df = df.withColumn("welfareFlag", df["welfareFlag"].cast('integer'))
    df = df.withColumn("welfareFlagDay", df["welfareFlagDay"].cast('integer'))
    df = df.withColumn("ZipCode", df["ZipCode"].cast('integer'))

    df = df.withColumn("DoW_Num", df["DoW_Num"].cast('integer'))
    df = df.withColumn("Total_SumQTY_Dow", df["Total_SumQTY_Dow"].cast('integer'))
    df = df.withColumn("AVG_SumQTY_Dow", df["AVG_SumQTY_Dow"].cast('double'))
    df = df.withColumn("MED_SumQTY_Dow", df["MED_SumQTY_Dow"].cast('double'))
    df = df.withColumn("SD_SumQTY_Dow", df["SD_SumQTY_Dow"].cast('double'))
    df = df.withColumn("NumWeekSale", df["NumWeekSale"].cast('integer'))
    df = df.withColumn("HighQualityFlag", df["HighQualityFlag"].cast('integer'))

    df = df.withColumn("MostSalesUnitFreq", df["MostSalesUnitFreq"].cast('double'))

    df = df.cache()

    return df

In [32]:
###########################################################################
############### Create Reference Quality Variable #########################
###########################################################################

reference_qty_df = spark.read.format("csv").option("header", "true").load(reference_qty)
reference_qty_df = reference_qty_df.withColumnRenamed("BranchCode","BranchRefQty").withColumnRenamed("MaterialCode","SKURefQty")

reference_qty_df = reference_qty_df.withColumn("BranchRefQty", reference_qty_df["BranchRefQty"].cast("integer"))
reference_qty_df = reference_qty_df.withColumn("SKURefQty", reference_qty_df["SKURefQty"].cast("integer"))

final_df = final_df.join(reference_qty_df,(final_df.BranchCode == reference_qty_df.BranchRefQty) &\
                         (final_df.MaterialCode == reference_qty_df.SKURefQty) &\
                          (final_df.DayofWeek == reference_qty_df.DoW_Num))

###########################################################################
############### Create High Quality Flag Variable #########################
###########################################################################

flag_qty_df = spark.read.format("csv").option("header", "true").load(flag_qty)
#flag_qty_df = flag_qty_df.filter(flag_qty_df.BranchCode.like('%M%') == False)

flag_qty_df = flag_qty_df.dropna()
flag_qty_df = flag_qty_df.withColumn("BranchCode", flag_qty_df["BranchCode"].cast("integer"))
flag_qty_df = flag_qty_df.withColumn("MaterialCode", flag_qty_df["MaterialCode"].cast("integer"))

l_bc = [cj[0] for cj in flag_qty_df.select('BranchCode').distinct().toLocalIterator()]
l_mc = [cj[0] for cj in flag_qty_df.select('MaterialCode').distinct().toLocalIterator()]

matches = (  (F.col('BranchCode').isin(l_bc)) & (F.col('MaterialCode').isin(l_mc))   )
final_df = final_df.withColumn("HighQualityFlag", when(matches, "1").otherwise("0"))

final_df = clean_binary_4(final_df)

In [33]:
# print('now features have: ', final_df.columns)
# print('now show of data: ', final_df.show(2))
# print('now count of data: ', final_df.count())

In [34]:
count_column_types(final_df)

type  count  \
0  bigint  1       
1  double  12      
2  int     19      
3  string  8       
4  vector  5       

                                                                                                                                                                                                                                                names  
0  cj_darkside_id                                                                                                                                                                                                                                      
1  supPrice | AVG_SumQTY_Dow | promotion_idx | ProvinceEN_idx | avgPriceDis | MostSalesUnitFreq | BranchCode_idx | SD_SumQTY_Dow | MaterialCode_idx | MED_SumQTY_Dow | DistrictEN_idx | avgPrice                                                       
2  MonthQuarter | DayofMonth | Month | ZipCode | NumWeekSale | WeekOfYear | Total_SumQTY_Dow | welfareFlag | welfareFlagDay | DayofWeek | BranchRefQty | DoW_Num | Yearday | MaterialCode | Year | BranchCode | HighQualityFlag | SKURefQty | Quarter  
3  DistrictEN | Name | MostSalesUnit | Branch | promotion | DoW | ProvinceEN | SalDate                                                                                                                                                                 
4  MaterialCode_vec | ProvinceEN_vec | promotion_vec | DistrictEN_vec | BranchCode_vec

In [35]:
# # check missing value
# final_df.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in final_df.columns)).show()

# Create 1) Data BKK

In [36]:
final_df.groupBy("ProvinceEN").agg(countDistinct("cj_darkside_id")).show(76)

+--------------------+---------------------+
|          ProvinceEN|count(cj_darkside_id)|
+--------------------+---------------------+
|          Ayutthaya |                  124|
|       Pathum Thani |                  143|
|       Prachin Buri |                   43|
|       Samut Prakan |                  311|
|         Ratchaburi |                  652|
|  Nakhon Ratchasima |                    3|
|            Bangkok |                  404|
|        Suphan Buri |                  380|
|          Chon Buri |                  334|
|       Chachoengsao |                   56|
|           Chumphon |                   15|
|Prachuap Khiri Khan |                  122|
|          Ang Thong |                   74|
|       Nakhon Nayok |                   19|
|           Lop Buri |                   35|
|           Chai Nat |                   25|
|       Kanchanaburi |                  399|
|         Nonthaburi |                  191|
|        Chanthaburi |                    2|
|       Na

In [37]:
df_bkk = final_df.filter( (final_df.ProvinceEN.like('%Bangkok%') == True) | (final_df.ProvinceEN.like('%Ratchaburi%') == True)  | (final_df.ProvinceEN.like('%Chon Buri%') == True)\
                         | (final_df.ProvinceEN.like('%Nonthaburi%') == True) | (final_df.ProvinceEN.like('%Nakhon Pathom%') == True)  | (final_df.ProvinceEN.like('%Samut Prakan%') == True)\
                         | (final_df.ProvinceEN.like('%Samut Sakhon%') == True))
#df_bkk.show(2)
df_bkk.cache()
df_bkk.count()

2822

In [38]:
df_bkk.groupBy("BranchCode").agg(countDistinct("cj_darkside_id")).count()

280

# Create 2) Data NOT BKK

In [39]:
df_not_bkk = final_df.filter( (final_df.ProvinceEN.like('%Bangkok%') == False) & (final_df.ProvinceEN.like('%Ratchaburi%') == False)  & (final_df.ProvinceEN.like('%Chon Buri%') == False)\
                         & (final_df.ProvinceEN.like('%Nonthaburi%') == False) & (final_df.ProvinceEN.like('%Nakhon Pathom%') == False)  & (final_df.ProvinceEN.like('%Samut Prakan%') == False)\
                         & (final_df.ProvinceEN.like('%Samut Sakhon%') == False))
#df_not_bkk.show(2)
df_not_bkk.cache()
df_not_bkk.count()

2179

In [40]:
df_not_bkk.groupBy("BranchCode").agg(countDistinct("cj_darkside_id")).count()

268

# GBT ML Model (Classifier)

In [41]:
############ ----- LOAD ML MODEL ----- ############

read_plmodelC_BKK = PipelineModel.read().load(PL_Bangkok_MODEL_Classification_PATH)
read_plmodelC_NotBKK = PipelineModel.read().load(PL_NotBangkok_MODEL_Classification_PATH)

print (read_plmodelC_BKK.stages)
print (read_plmodelC_NotBKK.stages)

############ ------------------- ----- ############

#read_model = GBTRegressor.read().load(ML_Bangkok_MODEL_Classification_PATH)

############ ----- DONE ML MODEL ----- ############

predictionsC_BKK = read_plmodelC_BKK.transform(df_bkk)
predictionsC_BKK.cache()

predictionsC_NotBKK = read_plmodelC_NotBKK.transform(df_not_bkk)
predictionsC_NotBKK.cache()

print("Testing Process: Done")

[VectorAssembler_3bb279e665f3, GBTClassificationModel: uid = GBTClassifier_b2b00292cd18, numTrees=5, numClasses=2, numFeatures=17]
[VectorAssembler_3bb279e665f3, GBTClassificationModel: uid = GBTClassifier_b2b00292cd18, numTrees=5, numClasses=2, numFeatures=17]
Testing Process: Done


In [42]:
predictionsC = predictionsC_BKK.union(predictionsC_NotBKK)
predictionsC.cache()

DataFrame[cj_darkside_id: bigint, SalDate: string, BranchCode: int, MaterialCode: int, avgPriceDis: double, avgPrice: double, supPrice: double, promotion: string, ZipCode: int, ProvinceEN: string, DistrictEN: string, DayofMonth: int, Yearday: int, Month: int, DayofWeek: int, Year: int, Quarter: int, WeekOfYear: int, MonthQuarter: int, Branch: string, Name: string, welfareFlag: int, welfareFlagDay: int, BranchCode_idx: double, BranchCode_vec: vector, MaterialCode_idx: double, MaterialCode_vec: vector, promotion_idx: double, promotion_vec: vector, ProvinceEN_idx: double, ProvinceEN_vec: vector, DistrictEN_idx: double, DistrictEN_vec: vector, BranchRefQty: int, SKURefQty: int, DoW: string, DoW_Num: int, Total_SumQTY_Dow: int, AVG_SumQTY_Dow: double, MED_SumQTY_Dow: double, SD_SumQTY_Dow: double, NumWeekSale: int, MostSalesUnit: string, MostSalesUnitFreq: double, HighQualityFlag: int, features_col: vector, rawPrediction: vector, probability: vector, predict_bi_lv1: double]

# GBT ML Model (Regressor)

In [43]:
############ ----- LOAD ML MODEL ----- ############

read_plmodelR_BKK = PipelineModel.read().load(PL_Bangkok_MODEL_Regression_PATH)
read_plmodelR_NotBKK = PipelineModel.read().load(PL_NotBangkok_MODEL_Regression_PATH)

print (read_plmodelR_BKK.stages)
print (read_plmodelR_NotBKK.stages)

############ ----- DONE ML MODEL ----- ############

predictionsR_BKK = read_plmodelR_BKK.transform(df_bkk)
predictionsR_BKK.cache()

predictionsR_NotBKK = read_plmodelR_NotBKK.transform(df_not_bkk)
predictionsR_NotBKK.cache()

print("Testing Process: Done")

[VectorAssembler_4420a6b41e11, GBTRegressionModel: uid=GBTRegressor_41320790faa8, numTrees=5, numFeatures=17]
[VectorAssembler_4420a6b41e11, GBTRegressionModel: uid=GBTRegressor_41320790faa8, numTrees=5, numFeatures=17]
Testing Process: Done


In [44]:
predictionsR = predictionsR_BKK.union(predictionsR_NotBKK)
predictionsR.cache()

DataFrame[cj_darkside_id: bigint, SalDate: string, BranchCode: int, MaterialCode: int, avgPriceDis: double, avgPrice: double, supPrice: double, promotion: string, ZipCode: int, ProvinceEN: string, DistrictEN: string, DayofMonth: int, Yearday: int, Month: int, DayofWeek: int, Year: int, Quarter: int, WeekOfYear: int, MonthQuarter: int, Branch: string, Name: string, welfareFlag: int, welfareFlagDay: int, BranchCode_idx: double, BranchCode_vec: vector, MaterialCode_idx: double, MaterialCode_vec: vector, promotion_idx: double, promotion_vec: vector, ProvinceEN_idx: double, ProvinceEN_vec: vector, DistrictEN_idx: double, DistrictEN_vec: vector, BranchRefQty: int, SKURefQty: int, DoW: string, DoW_Num: int, Total_SumQTY_Dow: int, AVG_SumQTY_Dow: double, MED_SumQTY_Dow: double, SD_SumQTY_Dow: double, NumWeekSale: int, MostSalesUnit: string, MostSalesUnitFreq: double, HighQualityFlag: int, features_col: vector, predict_qty_lv2: double]

# FINAL PREDICTION (C + R)

In [45]:
predictionsC.show(2)

+--------------+----------+----------+------------+-----------+--------+--------+---------+-------+-----------+----------+----------+-------+-----+---------+----+-------+----------+------------+-------------------+--------------------+-----------+--------------+--------------+-----------------+----------------+----------------+-------------+-------------+--------------+--------------+--------------+----------------+------------+---------+---+-------+----------------+--------------+--------------+-------------+-----------+-------------+-----------------+---------------+--------------------+--------------------+--------------------+--------------+
|cj_darkside_id|   SalDate|BranchCode|MaterialCode|avgPriceDis|avgPrice|supPrice|promotion|ZipCode| ProvinceEN|DistrictEN|DayofMonth|Yearday|Month|DayofWeek|Year|Quarter|WeekOfYear|MonthQuarter|             Branch|                Name|welfareFlag|welfareFlagDay|BranchCode_idx|   BranchCode_vec|MaterialCode_idx|MaterialCode_vec|promotion_idx|pro

In [46]:
predictionsR.show(2)

+--------------+----------+----------+------------+-----------+--------+--------+---------+-------+-----------+----------+----------+-------+-----+---------+----+-------+----------+------------+-------------------+--------------------+-----------+--------------+--------------+-----------------+----------------+----------------+-------------+-------------+--------------+--------------+--------------+----------------+------------+---------+---+-------+----------------+--------------+--------------+-------------+-----------+-------------+-----------------+---------------+--------------------+------------------+
|cj_darkside_id|   SalDate|BranchCode|MaterialCode|avgPriceDis|avgPrice|supPrice|promotion|ZipCode| ProvinceEN|DistrictEN|DayofMonth|Yearday|Month|DayofWeek|Year|Quarter|WeekOfYear|MonthQuarter|             Branch|                Name|welfareFlag|welfareFlagDay|BranchCode_idx|   BranchCode_vec|MaterialCode_idx|MaterialCode_vec|promotion_idx|promotion_vec|ProvinceEN_idx|ProvinceEN_v

In [47]:
def ansByProb(p, t=0.4):
  if p>=t: return 1
  elif p<t: return 0
  else: 0

predictionsA = predictionsC.join(predictionsR,(predictionsC.SalDate == predictionsR.SalDate)&\
                  (predictionsC.BranchCode == predictionsR.BranchCode)&\
                  (predictionsC.MaterialCode == predictionsR.MaterialCode)).select(predictionsC["*"],predictionsR["predict_qty_lv2"])

split1_udf = udf(lambda value: value[0].item(), FloatType())
split2_udf = udf(lambda value: value[1].item(), FloatType())

predictionsA = predictionsA.withColumn('probability_class0', split1_udf('probability')).withColumn('probability_class1', split2_udf('probability'))

ansByP = F.udf(ansByProb, IntegerType())
predictionsA = predictionsA.withColumn("predict_bi_lv1_with_thresh", ansByP("probability_class1"))

In [48]:
predictionsA = predictionsA.withColumn("predict_qty_final", F.ceil(col("predict_bi_lv1")*col("predict_qty_lv2")))
#predictionsA = predictionsA.withColumn("predict_qty_final", col("predict_bi_lv1_with_thresh")*col("predict_qty_lv2"))

In [51]:
predictionsA.select(['SalDate','BranchCode','MaterialCode','AVG_SumQTY_Dow',\
                    'predict_bi_lv1','predict_qty_lv2',\
                     'predict_qty_final']).show(50)

+----------+----------+------------+--------------+--------------+------------------+-----------------+
|   SalDate|BranchCode|MaterialCode|AVG_SumQTY_Dow|predict_bi_lv1|   predict_qty_lv2|predict_qty_final|
+----------+----------+------------+--------------+--------------+------------------+-----------------+
|2021-10-03|         2|    20029778|         18.05|           1.0|27.891596281415648|               28|
|2021-10-03|         2|    20006553|          5.33|           1.0|3.7047673513115735|                4|
|2021-10-03|         2|    20030397|         16.93|           1.0|27.891596281415648|               28|
|2021-10-03|         2|    20006489|         80.25|           1.0| 93.33532627766304|               94|
|2021-10-03|         2|    20026655|          65.7|           0.0|100.44230364966938|                0|
|2021-10-03|         8|    20025122|         11.18|           1.0|29.438285296902553|               30|
|2021-10-03|         8|    20006978|         55.35|           1.

# POST PROCESSING (WITH TIME SERIES)

In [52]:
# predictionsA = post_score_v3(predictionsA)
# predictionsA.show(2)

In [53]:
'Hi'

'Hi'

# Criteria 
![](https://github.com/kaocj/demandforecasting_darkside/raw/main/criteria.PNG)

In [54]:
'Hi'

'Hi'